# Код для создания видеосэмплов отдельных балетных па
Видеосэмплы используют для предпросмотра балетных па из Библиотеки балетных па.

Видеосэмплы созданы на основе кадров (фреймов) из Библиотеки балетных па (созданной в процессе выполнения проекта)

Для экономии памяти, видеосэмплы сделаны в уменьшенном формате: **256 х 144** пикселей

## Алгоритм создания видеосэмплов

1. Напишем функцию **get_frames**: для получения списка кадров, из которых состоит балетный элемент
2. Напишем функцию **generate_video**: для генерирования видеосэмплов из списка кадров
3. Импортируем необходимые библиотеки
4. Загружаем excel-файл и формируем два датафрейма: **elements** и **frame_index**
5. Генерируем видеосэмплы и сохраняем их в соответствующие папки

### 1. Функция для получения списка кадров, из которых состоит балетный элемент

+  создаем функцию get_frames, которая по индентификатору балетного элемента (id_element) выдает список фреймов (кадров), из которых состоит балетный элемент

**id_element** состоит из трех частей: `Элемент_Балет_Num`
+ название элемента
+  название балета (балетного номера)
+ порядковый номер этого элемента в этом балетном номере среди аналогичных элементов

In [1]:
def get_frames(id_element):
    """
    Функция для получения списка кадров (названия jpg файлов), из которых состоит балетный элемент.

    Параметры:
    -----------
    id_element: str
        Название балетного элемента в формате "Элемент_Балет_Номер"

    """
    # определяем название первого кадра, из которого состоит конкретный балетный элемент
    first_frame = elements[elements['id_element'] == id_element]['Begin'].values[0]

    # добавляем к названию тип файла (расширение)
    first_frame_jpg = first_frame + '.jpg'

    # определяем название последнего кадра, из которого состоит конкретный балетный элемент
    last_frame = elements[elements['id_element'] == id_element]['End'].values[0]
    last_frame_jpg = last_frame + ".jpg"

    # индекс первого кадра
    index_begin = frame_index[frame_index['frames']==first_frame_jpg].index[0] - 5

    # индекс последнего кадра
    index_end = frame_index[frame_index['frames']==last_frame_jpg].index[0] + 5

    # формируем список из индексов кадров
    # в список входит индекс первого кадра, последнего кадра и всех промежуточных кадров
    frames_list = list(range(index_begin, index_end))

    # по списку из индексов отбираем названия кадров, которые формируют наш id_element
    frames = frame_index[frame_index.index.isin(list(range(index_begin, index_end+1)))]

    # формируем список из кадров, которые относятся к данному элементу
    pictures = list(frame_index[frame_index.index.isin(list(range(index_begin,index_end)))]['frames'])

    return pictures


### 2. Функция для генерирования видеосэмплов из списка кадров
+ создаем функцию generate_video, которая генерирует видеофайл из набора (списка) фреймов (кадров): для каждого балетного элемента свой видеофайл 

In [2]:
def generate_video(id_element, samples_folder, image_folder='BalletFramesSmall'):
    """
    Функция для генерирования видеосэмплов из списка кадров.
    По умолчанию формат видео: 256 на 144

    Параметры:
    + id_element: string,
        Идентификатор балетного элемента
    + samples_folder: string
        Папка, в которую сохраняются видеосэмплы
    + image_folder: string
        Папка с кадрами (уменьшенными)

    """
    width=256
    height=144
    print(f'{id_element} sample ')

    # указываем, как будут называться файлы-видеосэмплы
    video_name = os.path.join(samples_folder, f'{id_element}.avi')

    # применяем нашу функцию get_frames
    # получаем список кадров, которые формируют конкретный балетный элемент
    images = get_frames(id_element)
    print(id_element)

    # создаем экземпляр VideoWriter
    # указываем параметры: название файла, кодек по умолчанию, частота кадров (25),
    video = cv2.VideoWriter(video_name, 0, 25, (width, height))

    # генерируем видео последовательно, добавляя кадры из нашего списка
    for image in images:
        print(os.path.join(image_folder, image))
        video.write(cv2.imread(os.path.join(image_folder, image)))

    # Очищаем память
    cv2.destroyAllWindows()
    video.release()



## Создание видео

### 3. Импортируем необходимые библиотеки

In [3]:
import pandas as pd # пандас - библиотека для обработки excel файлов
import cv2 # open cv - библиотека для получения видео из набора кадров (фреймов)
import os # библиотека для работы с именами файлов и их расположением

from PIL import Image # для изменения размера кадров

### 4. Загружаем excel-файл и формируем два датафрейма

In [4]:
# загружаем датафреймы
file = "Create_Ballet_v1.3.xlsx"

elements = pd.read_excel(file, 
                         sheet_name="Элементы_экземпляры_кадры",
                        header=0)

In [ ]:
# смотрим первые 5 строк
elements.head()

In [ ]:
elements.tail()

In [7]:
frame_index = pd.read_excel(file, 
                            sheet_name="кадры_индексы",
                            index_col=0,
                            usecols=['index', 'frames'])
frame_index.head()

,frames
index,
0,Baryshnikov_0-00-00.00.jpg
1,Baryshnikov_0-00-00.04.jpg
2,Baryshnikov_0-00-00.08.jpg
3,Baryshnikov_0-00-00.12.jpg
4,Baryshnikov_0-00-00.16.jpg


### 5. Генерируем видеосэмплы и сохраняем их в соответствующие (дочерние) папки

In [ ]:
# Основная (родительская) папка для видеосэмплов
folder_parent = "BalletVideoSamples"



# перебираем все строки (балетные элементы) в датафрейме elements
for idx in elements.index:

    # название дочерних папок - папок для каждой группы элементов
    # эти папки находятся внутри родительской папки "BalletVideoSamples"
    folder_element = os.path.join(folder_parent,
                                  elements.loc[idx, 'Элемент'])

    # проверяем наличие дочерней папки (папки для отдельной группы элементов)
    # если дочерней папки не существует внутри родительской - то создаем её
    if not os.path.isdir(folder_element):
        os.mkdir(folder_element)

    # создаем видеосэмплы с помощью нашей функции generate_video
    # каждый видеосэмпл сохраняем в соответствующую дочернюю папку
    id_element = elements.loc[idx, 'id_element']
    generate_video(id_element, folder_element)

print('Готово!')